In [39]:
import pandas as pd


In [40]:
from sklearn.preprocessing import OneHotEncoder
df = pd.read_csv("processed_dataset.csv")

#one hot for loan_grade_encoded
one_hot_encoder = OneHotEncoder(
    categories="auto",
    sparse_output=False
)
one_hot_array = one_hot_encoder.fit_transform(df[["loan_grade"]])
one_hot_df = pd.DataFrame(one_hot_array, columns=one_hot_encoder.get_feature_names_out(["loan_grade"]))
one_hot_df = one_hot_df.astype(bool)
one_hot_df.describe(include="all")

,loan_grade_A,loan_grade_B,loan_grade_C,loan_grade_D,loan_grade_E,loan_grade_F,loan_grade_G
count,31743,31743,31743,31743,31743,31743,31743
unique,2,2,2,2,2,2,2
top,False,False,False,False,False,False,False
freq,21215,21576,25450,28215,30813,31509,31680


In [45]:
# Sadece egitim ve tahminleme islemlerinde gereken column'lar aliniyor.
df = pd.concat([
    df,
    one_hot_df
], axis=1)

train_cols = [
    "log_income",
    "log_person_age",
    "loan_intent_VENTURE",
    "loan_intent_PERSONAL",
    "loan_intent_MEDICAL",
    "loan_intent_HOMEIMPROVEMENT",
    "loan_intent_EDUCATION",
    "person_home_ownership_OWN",
    "person_home_ownership_RENT",
    "cb_person_default_on_file",
    "loan_percent_income"
]

predict_cols = [
    "loan_grade_A",
    "loan_grade_B",
    "loan_grade_C",
    "loan_grade_D",
    "loan_grade_E",
    "loan_grade_F",
    "loan_grade_G",
]


df = df[[*predict_cols, *train_cols]]

In [46]:
summary = pd.DataFrame({
    'min': df.min(),
    'max': df.max(),
    'mean': df.mean()
})
print(summary)

                                  min       max          mean
loan_grade_A                    False      True  3.316637e-01
loan_grade_A                    False      True  3.316637e-01
loan_grade_B                    False      True  3.202911e-01
loan_grade_B                    False      True  3.202911e-01
loan_grade_C                    False      True  1.982484e-01
loan_grade_C                    False      True  1.982484e-01
loan_grade_D                    False      True  1.111426e-01
loan_grade_D                    False      True  1.111426e-01
loan_grade_E                    False      True  2.929780e-02
loan_grade_E                    False      True  2.929780e-02
loan_grade_F                    False      True  7.371704e-03
loan_grade_F                    False      True  7.371704e-03
loan_grade_G                    False      True  1.984690e-03
loan_grade_G                    False      True  1.984690e-03
log_income                  -2.772662  2.778184 -2.492708e-15
log_pers

In [48]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Burada standardize etmek istedigimiz column'lari seciyoruz. (surekli degerler)
cont_cols = [
    "log_income",
    "log_person_age",
    "loan_percent_income"
]

target_df = df[cont_cols]
df = df.drop(columns=cont_cols)
scaler.fit(target_df)
target_df = pd.DataFrame(
    scaler.transform(target_df),
    columns=target_df.columns
)
df = pd.concat([
    target_df,
    df
], axis=1)

In [49]:
summary = pd.DataFrame({
    'min': df.min(),
    'max': df.max(),
    'mean': df.mean()
})
print(summary)

                                  min       max          mean
log_income                  -2.772662  2.778184 -5.730364e-17
log_person_age              -1.551861  6.640501  0.000000e+00
loan_percent_income         -1.515502  6.231904  1.790739e-17
loan_grade_A                    False      True  3.316637e-01
loan_grade_A                    False      True  3.316637e-01
loan_grade_B                    False      True  3.202911e-01
loan_grade_B                    False      True  3.202911e-01
loan_grade_C                    False      True  1.982484e-01
loan_grade_C                    False      True  1.982484e-01
loan_grade_D                    False      True  1.111426e-01
loan_grade_D                    False      True  1.111426e-01
loan_grade_E                    False      True  2.929780e-02
loan_grade_E                    False      True  2.929780e-02
loan_grade_F                    False      True  7.371704e-03
loan_grade_F                    False      True  7.371704e-03
loan_gra

In [50]:
from sklearn.model_selection import train_test_split
# Test ve egitim verisini %90'a %10 oranla ayiriyoruz
train_data, test_data = train_test_split(df, test_size=0.1, train_size=0.9)

In [51]:
# Dogru bir sekilde ayirilmis mi diye bakiyoruz
len(train_data) / (len(train_data) + len(test_data))

0.8999779478940239

In [52]:
# Emin oldugumuza gore, egitim ve test datalarini uygun sekilde input ve output olarak boluyoruz
train_X = train_data[train_cols]
train_y = train_data[predict_cols]

test_X = test_data[train_cols]
test_y = test_data[predict_cols]